Pobieranie wiadomości z naszdziennik.pl Zapis do pliku ./data/naszdziennik_{timestamp}.txt

In [1]:
# ! pip install bs4
# ! pip install requests
# ! pip install tqdm
# ! pip install pandas
# ! pip install pyarrow
# ! pip install qgrid
# ! pip install blaze
# ! pip install ipynbname
# ! pip install certifi

In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm, trange
from datetime import datetime, timezone
from common.webarticle import WebArticle
from common.pl_helper import PolishLocaleHelper
import pandas as pd
import logging
import ipynbname
from urllib3.exceptions import InsecureRequestWarning


In [3]:
# settings:
portal_name = 'naszdziennik'
website = 'https://naszdziennik.pl/polska'
website_short = 'https://naszdziennik.pl'
pagination_attributes = '?s='
no_articles_to_scrap = 300
data_dir = '../data'
logging.basicConfig(
    filename=f'../logs/{ipynbname.name()}.ipynb.log',
    encoding='utf-8',
    level=logging.DEBUG
)
ignored_paragraph_starts = []

Sprawdźmy plik robots.txt, niestety strona naszdziennik takiego nie posiada, trudno odnaleźć również regulamin strony

In [4]:
# resp = requests.get('https://www.naszdziennik.pl/robots.txt')
# print(resp.text)

In [5]:
def check_if_ignored_paragraph(text, ignored_paragraphs, operation='startswith'):

    for ignored_paragraph in ignored_paragraphs:
        if operation == 'startswith':
            if text.startswith(ignored_paragraph):
                return True
        else:
            logging.debug(f'check_if_ignored: operation {operation} not implemented')
    return False


In [6]:
def scrap_article_text_naszdziennik(link):
    logging.info('scrap_article_text_naszdziennik:')
    logging.info(f'article link to scrap: {link}')

    requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)
    page = requests.get(link, verify=False)
    if page.status_code != 200:
        logging.debug(f'received status code: {page.status_code}')
        return '', ''

    soup = BeautifulSoup(page.content, 'html.parser')
    paragraphs = soup.find('div', id='article-content')

    text = ''
    if paragraphs is None:
        # paywall - load just part of article
        lead_text_part = soup.find('div', id='content').find('p').find('strong')
        if lead_text_part is None:
            lead_text_int = soup.find('div', id='article').find('h2').get_text().strip()
        else:
            lead_text_int = lead_text_part.get_text().strip()
            lead_text_part.decompose()
        text = soup.find('div', id='article').find('div', id='content', class_='content').get_text(strip=True)
        time_string = soup.find('div', id='header', class_='hidden-xs').get_text(strip=True)
        when_published = PolishLocaleHelper.parse_pl_date_naszdziennik(time_string[:time_string.rfind(',')], short=True)

        author = soup.find('p', class_='author').get_text(strip=True)
        source = 'NaszDziennik.pl'

    else:
        paragraphs = paragraphs.find_all('p')
        for paragraph in paragraphs:
            paragraph_text = paragraph.get_text().strip()
            if check_if_ignored_paragraph(paragraph_text, ignored_paragraph_starts):
                continue
            text += paragraph_text + ' '
        time_string = soup.find('div', id='article-date').get_text(strip=True)
        when_published = PolishLocaleHelper.parse_pl_date_naszdziennik(time_string)
        lead_text_int = soup.find('div', id='article-subtitle').get_text().strip()
        author = ''
        author_candidate = soup.find('p', id='article-author')
        if author_candidate is not None:
            author = author_candidate.get_text(strip=True)

        source_candidate = soup.find('div', id='article').find('p', id='article-source')
        if source_candidate.get_text(strip=True) == '':
            source = source_candidate.find('img', alt=True)['alt']
        else:
            source = source_candidate.get_text(strip=True)

    title_int = soup.find('div', id='article').find('h1').get_text().strip()

    return text, when_published, title_int, lead_text_int, author, source

In [7]:
def scrap_articles_naszdziennik(website):
    logging.info('scrap_articles_naszdziennik:')
    logging.info(f'page with articles to scrap: {website}')

    result = list()
    requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)
    page = requests.get(website, verify=False)
    if page.status_code != 200:
        logging.debug(f'received status code: {page.status_code}')
        return result

    soup = BeautifulSoup(page.content, 'html.parser')
    page_content = soup.find('div',id='article-list').find_all('div',class_='article')

    print(f'articles to scrap: {len(page_content)}')

    for article in page_content:

        logging.info(f"article scrapping started: {article}")
        title = article.find('h2').get_text().strip()
        lead_text = article.find('div', class_='intro').get_text().strip()
        link = website_short + article.find('a', href=True)['href']
        webart = WebArticle(title, lead_text, link, *scrap_article_text_naszdziennik(link),portal_name)
        logging.info(f'webarticle scrapped: {webart}')
        result.append(webart)

    return result

In [8]:
no_scrapped_articles = 0
page_no=1
scrapped_articles = list()

logging.info(f'no_articles_to_scrap: {no_articles_to_scrap}')

pbar = tqdm(total=no_articles_to_scrap)
pbar.update(0)
while no_scrapped_articles < no_articles_to_scrap:
    logging.info('next page: '+website+pagination_attributes+str(page_no))
    new_articles = scrap_articles_naszdziennik(website+pagination_attributes+str(page_no))
    page_size = len(new_articles)
    logging.info(f'new_articles: {page_size}')
    if no_scrapped_articles + page_size > no_articles_to_scrap:
        page_size = no_articles_to_scrap - no_scrapped_articles
    scrapped_articles.extend(new_articles[:page_size])
    logging.info(f'scrapped new articles: {page_size}')
    no_scrapped_articles += page_size
    pbar.update(page_size)
    logging.info(f'no_scrapped_articles (total): {no_scrapped_articles}')
    page_no += 1
    if len(scrapped_articles) <= 0:
        logging.debug(f'0 articles scrapped!')
        break
pbar.close()

  0%|          | 0/300 [00:00<?, ?it/s]

articles to scrap: 8


  3%|▎         | 8/300 [00:08<04:53,  1.01s/it]

articles to scrap: 8


  5%|▌         | 16/300 [00:16<05:02,  1.07s/it]

...

 96%|█████████▌| 288/300 [05:23<00:13,  1.12s/it]

articles to scrap: 8


 99%|█████████▊| 296/300 [05:32<00:04,  1.13s/it]

articles to scrap: 8


100%|██████████| 300/300 [05:42<00:00,  1.14s/it]


In [9]:
for i in trange(len(scrapped_articles)):
    # print(f'{i}: {scrapped_articles[i].lead_text}')
    print(f'{i}: {len(scrapped_articles[i].text)}')

100%|██████████| 300/300 [00:00<00:00, 11999.50it/s]

0: 1115
1: 1422
2: 4762
3: 5356
4: 8874
5: 3455
6: 25725
7: 455
8: 479
9: 1473
10: 8659
11: 0
12: 552
13: 3331
14: 25684
15: 1099
16: 392
17: 852
18: 215
19: 2376
20: 0
21: 984
22: 2796
23: 1750
24: 25642
25: 934
26: 243
27: 215
28: 1516
29: 3257
30: 0
31: 25600
32: 838
33: 973
34: 1180
35: 1671
36: 2023
37: 9360
38: 25558
39: 3486
40: 1764
41: 0
42: 1897
43: 2083
44: 25519
45: 1005
46: 162
47: 1704
48: 7488
49: 2362
50: 964
51: 0
52: 3499
53: 1558
54: 1317
55: 25477
56: 222
57: 2820
58: 0
59: 4902
60: 895
61: 9532
62: 25449
63: 1041
64: 805
65: 0
66: 9379
67: 1265
68: 891
69: 25407
70: 460
71: 888
72: 1373
73: 200
74: 132
75: 0
76: 7590
77: 708
78: 1481
79: 5585
80: 25363
81: 2174
82: 536
83: 889
84: 294
85: 25309
86: 0
87: 1501
88: 2222
89: 25267
90: 425
91: 1473
92: 141
93: 0
94: 4044
95: 6165
96: 3596
97: 25226
98: 1685
99: 673
100: 713
101: 103
102: 1240
103: 25185
104: 0
105: 1451
106: 1424
107: 2288
108: 5658
109: 1429
110: 25158
111: 2060
112: 457
113: 591
114: 0
115: 3656
116:

In [10]:
naszdziennik_dataframe = pd.DataFrame(article.__dict__ for article in scrapped_articles)
naszdziennik_dataframe

C:\Users\piotr\AppData\Local\Temp/ipykernel_24080/3830852082.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  naszdziennik_dataframe = pd.DataFrame(article.__dict__ for article in scrapped_articles)


,title,title_int,lead_text,lead_text_int,link,text,when_published,author,source,portal
0,Szybszy powrót do zdrowia,Szybszy powrót do zdrowia,Rehabilitacja pocovidowa zdecydowanie przyspie...,Rehabilitacja pocovidowa zdecydowanie przyspie...,"https://naszdziennik.pl/polska-kraj/249411,szy...",Rehabilitacja osób po covidzie odbywa się w po...,2022-01-20 00:03:00,Marek Zygmunt,Nasz Dziennik,naszdziennik
1,Komunikat IPN,Komunikat IPN,Akt oskarżenia przeciwko byłemu prokuratorowi ...,19 stycznia 2022 r. Oddziałowa Komisja Ścigani...,"https://naszdziennik.pl/polska-kraj/249391,kom...",Podejrzanemu zarzucono popełnienie czynów pole...,2022-01-19 17:09:00,"przesł. Janusz Ślęzak, Oddział IPN w Krakowie",NaszDziennik.pl,naszdziennik
2,Spotkanie dyrektorów Caritas,Spotkanie dyrektorów Caritas,"Pomoc dzieciom, seniorom, migrantom – Caritas ...","O dalszej pomocy dzieciom, seniorom, wykluczon...","https://naszdziennik.pl/polska-kraj/249387,spo...",W obradach uczestniczył m.in. ks. bp Wiesław S...,2022-01-19 16:34:00,"APW, KAI",NaszDziennik.pl,naszdziennik
3,Zabytkowy nagrobek odrestaurowany,Zabytkowy nagrobek odrestaurowany,Uroczystość poświęcenia grobu weterana Powstan...,21 stycznia 2022 r. w Nowym Mieście Lubawskim...,"https://naszdziennik.pl/polska-kraj/249382,zab...",Program 21 stycznia 2022 (piątek) godz. 12.00 ...,2022-01-19 14:56:00,"APW, https://gdansk.ipn.gov.pl",NaszDziennik.pl,naszdziennik
4,Falstart „Polskiego ładu”,Falstart „Polskiego ładu”,O błędach i niedociągnięciach rozwiązań „Polsk...,Z dr. Tomaszem Bojkowskim z Katedry Finansów P...,"https://naszdziennik.pl/polska-kraj/249380,fal...",– Nie najlepiej. Moim zdaniem aktualne rozwiąz...,2022-01-19 14:32:00,Mariusz Kamieniecki,NaszDziennik.pl,naszdziennik
...,...,...,...,...,...,...,...,...,...,...
295,W czwartek w „Naszym Dzienniku”,W czwartek w „Naszym Dzienniku”,"Chcesz wiedzieć, co znajdziesz w „Naszym Dzien...","Chcesz wiedzieć, co znajdziesz w „Naszym Dzien...","https://naszdziennik.pl/polska-kraj/247495,w-c...",,2021-12-08 18:43:00,"APW, PAP",NaszDziennik.pl,naszdziennik
296,"Agresora trzeba powstrzymać, zanim urośnie w siłę","Agresora trzeba powstrzymać, zanim urośnie w siłę","O rozmowach Biden – Putin, zbyt miękkiej polit...","Z gen. dyw. Romanem Polką, byłym dowódcą Jedno...","https://naszdziennik.pl/polska-kraj/247492,agr...",Tak naprawdę skutki czy efekty tej rozmowy dop...,2021-12-08 18:27:00,Mariusz Kamieniecki,NaszDziennik.pl,naszdziennik
297,Jak kształtuje się liczba zakażonych,Jak kształtuje się liczba zakażonych,"Dowiedz się, jak przyrasta codziennie liczba z...","Dowiedz się, jak przyrasta codziennie liczba z...","https://naszdziennik.pl/polska-kraj/247482,jak...",Od początku epidemii z powodu zakażenia COVID-...,2021-12-08 14:04:00,APW,NaszDziennik.pl,naszdziennik
298,Dodatkowe wsparcie z kuratorium,Dodatkowe wsparcie z kuratorium,"ROZMOWA z Barbarą Nowak, małopolską kurator oś...","ROZMOWA z Barbarą Nowak, małopolską kurator oś...","https://naszdziennik.pl/polska-kraj/247472,dod...","– Rozumiem, że chodzi o troskę ministra zdrowi...",2021-12-08 09:05:00,Urszula Wróbel,Nasz Dziennik,naszdziennik


In [11]:
timestamp = int((datetime.now(timezone.utc)).timestamp())

In [12]:
naszdziennik_dataframe.to_parquet(f'{data_dir}/{portal_name}_{timestamp}.parquet')
naszdziennik_dataframe.to_json(f'{data_dir}/{portal_name}_{timestamp}.json')

Krótki test - dane pierwszego artykułu

In [13]:
print(f'title:\n  {naszdziennik_dataframe.iloc[0]["title"]}')
print(f'lead_text:\n  {naszdziennik_dataframe.iloc[0]["lead_text"]}')
print(f'lead_text_int:\n  {naszdziennik_dataframe.iloc[0]["lead_text_int"]}')
print(f'link:\n  {naszdziennik_dataframe.iloc[0]["link"]}')
print(f'text:\n  {naszdziennik_dataframe.iloc[0]["text"]}')
print(f'when_published:\n  {naszdziennik_dataframe.iloc[0]["when_published"]}')

title:
  Szybszy powrót do zdrowia
lead_text:
  Rehabilitacja pocovidowa zdecydowanie przyspiesza powrót pacjenta do normalnego funkcjonowania
lead_text_int:
  Rehabilitacja pocovidowa zdecydowanie przyspiesza powrót pacjenta do normalnego codziennego funkcjonowania, a także do pracy zawodowej. Twierdzą tak zarówno lekarze, jak i osoby, które przeszły lub przechodzą leczenie pocovidowe.
link:
  https://naszdziennik.pl/polska-kraj/249411,szybszy-powrot-do-zdrowia.html
text:
  Rehabilitacja osób po covidzie odbywa się w podmiotach leczniczych, które są zakładami lecznictwa uzdrowiskowego, lub w ośrodkach realizujących taką rehabilitację w trybie stacjonarnym, z odpowiednią bazą zabiegową. Uzdrowisko Cieplice było jednym z pierwszych w naszym kraju, które wprowadziło do swojej oferty rehabilitację osób, które przeszły COVID-19. – Wykorzystując posiadany potencjał kadrowo-lokalowo-sprzętowy w zakresie leczenia uzdrowiskowego i rehabilitacji leczniczej, już w grudniu 2020 r. na podstawie au

In [14]:
print(f'title:\n  {naszdziennik_dataframe.iloc[99]["title"]}')
print(f'lead_text:\n  {naszdziennik_dataframe.iloc[99]["lead_text"]}')
print(f'link:\n  {naszdziennik_dataframe.iloc[99]["link"]}')
print(f'text:\n  {naszdziennik_dataframe.iloc[99]["text"]}')
print(f'when_published:\n  {naszdziennik_dataframe.iloc[99]["when_published"]}')

title:
  Zbudźmy sumienia
lead_text:
  Aborcja była główną przyczyną śmierci na świecie w ubiegłym roku
link:
  https://naszdziennik.pl/polska-kraj/248771,zbudzmy-sumienia.html
text:
  Według strony „Worldometers”, pokazującej statystyki dotyczące różnych kwestii społecznych, oficjalnie do 31.12.2021 r. przeprowadzono 42 mln 640 tys. 209 aborcji. Dane mogą być wyższe ze względu na aborcje dokonywane poza warunkami szpitalnymi oraz na dostęp do środków wczesnoporonnych. Oficjalnie na świecie zmarło 58,7 mln ludzi. Równocześnie wiele krajów zmaga się z gwałtownym starzeniem i wymieraniem społeczeństw, wprowadza programy mające poprawić demografię. Drogi Czytelniku, o problemach demograficznej zapaści i śmierci poza statystykami możesz przeczytać w dzisiejszym papierowym wydaniu „Naszego Dziennika”, dostępnym w punktach sprzedaży prasy, oraz TUTAJ. 
when_published:
  2022-01-07 09:03:00


In [16]:
# Fix 3 timestamps issues (multi-day-dates) for 141 (2021-12-31), 235 (2021-12-18) and 279 (2021-12-11)
naszdziennik_dataframe.at[141,'when_published'] = datetime.strptime('2021-12-31', "%Y-%m-%d")
naszdziennik_dataframe.at[235,'when_published'] = datetime.strptime('2021-12-18', "%Y-%m-%d")
naszdziennik_dataframe.at[279,'when_published'] = datetime.strptime('2021-12-11', "%Y-%m-%d")

naszdziennik_dataframe.to_parquet(f'{data_dir}/{portal_name}_{timestamp}.parquet')
naszdziennik_dataframe.to_json(f'{data_dir}/{portal_name}_{timestamp}.json')